In [1]:
import nltk
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
from TurkishStemmer import TurkishStemmer
from gensim.models import FastText

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
#Upload Data
data = pd.read_excel("dataset.xlsx",header=None)

data.columns = ['Sentence','Class']
data

,Sentence,Class
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz
...,...,...
41177,İnternet kotasının bitmesinden.,Ambigious
41178,Biri bana bir işi nasıl yapmam gerektiğini söy...,Anger
41179,Park halindeki arabama çarptıklarında.,Anger
41180,Bardağımı başkasının kullanması.,Disgust


In [3]:
data.head()

,Sentence,Class
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz


In [4]:
# data preprocessing
data['Sentence'] = data['Sentence'].str.strip().str.lower()

data["Class"].unique()

#remove email adress
data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)

#remove IP address
data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)

#remove punctaitions and special chracters
data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')

#remove numbers
data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)

C:\Users\Ersu\AppData\Local\Temp/ipykernel_1412/789653276.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')


In [5]:
data.head()

,Sentence,Class
0,ulan wifiye bağlıyım ben ona bağlıyken turkcel...,olumsuz
1,dk gb internet mb sadece kaşar turkcell de ...,olumsuz
2,ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,turkcell çok pahalı ya,olumsuz
4,turkcell kaşta internetin cekmiyor,olumsuz


In [6]:
data.isnull().sum()

Sentence    0
Class       0
dtype: int64

In [7]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models import FastText

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ersu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
for i, cell in enumerate(data["Class"]):
    if cell == "olumsuz":
        value = 0
    elif cell == "olumlu":
        value = 1
    elif cell == "Angry":
        value = 0
    elif cell == "Happy":
        value = 1
    elif cell == "Fear":
        value = 0
    elif cell == "Sadness":
        value = 0
    elif cell == "Surprise":
        value = 1
    elif cell == "Disgust":
        value = 0
    elif cell == "Ambigious":
        value = 1
    else:
        value = 2
    data["Class"].loc[i] = value

# BAG OF WORD

In [9]:
bOw_vectorizer = CountVectorizer(max_features=1000)
bOw_matrix = bOw_vectorizer.fit_transform(data["Sentence"].tolist()).toarray()

In [10]:
X = bOw_matrix
y = data["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train.tolist(), y_train.tolist())

GaussianNB()

In [12]:
pred = model.predict(X_test.tolist())

In [13]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))


              precision    recall  f1-score   support

           0       0.89      0.50      0.64      4641
           1       0.75      0.60      0.66      4479
           2       0.28      0.98      0.43      1176

    accuracy                           0.60     10296
   macro avg       0.64      0.69      0.58     10296
weighted avg       0.76      0.60      0.63     10296



In [14]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train.tolist(), y_train.tolist())

DecisionTreeClassifier()

In [15]:
pred = model.predict(X_test.tolist())

In [16]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.72      0.77      0.74      4641
           1       0.76      0.72      0.74      4479
           2       0.74      0.67      0.70      1176

    accuracy                           0.74     10296
   macro avg       0.74      0.72      0.73     10296
weighted avg       0.74      0.74      0.74     10296



In [17]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(X_train.tolist(), y_train.tolist())

RandomForestClassifier()

In [18]:
pred = model.predict(X_test.tolist())

In [19]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.77      0.77      0.77      4641
           1       0.77      0.78      0.78      4479
           2       0.76      0.70      0.73      1176

    accuracy                           0.77     10296
   macro avg       0.77      0.75      0.76     10296
weighted avg       0.77      0.77      0.77     10296



# TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_features=1000)
tf_idf_matrix = tf_idf_vectorizer.fit_transform(data["Sentence"].tolist()).toarray()

In [21]:
X = tf_idf_matrix
y = data["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train.tolist(), y_train.tolist())

GaussianNB()

In [23]:
pred = model.predict(X_test.tolist())

In [24]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.94      0.05      0.09      4606
           1       0.73      0.03      0.06      4508
           2       0.12      1.00      0.21      1182

    accuracy                           0.15     10296
   macro avg       0.60      0.36      0.12     10296
weighted avg       0.75      0.15      0.09     10296



In [25]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train.tolist(), y_train.tolist())

DecisionTreeClassifier()

In [26]:
pred = model.predict(X_test.tolist())

In [27]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.46      0.98      0.62      4606
           1       0.66      0.04      0.08      4508
           2       0.68      0.07      0.13      1182

    accuracy                           0.46     10296
   macro avg       0.60      0.36      0.28     10296
weighted avg       0.57      0.46      0.33     10296



In [28]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(X_train.tolist(), y_train.tolist())

RandomForestClassifier()

In [29]:
pred = model.predict(X_test.tolist())

In [30]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.46      0.97      0.62      4606
           1       0.66      0.05      0.09      4508
           2       0.67      0.08      0.14      1182

    accuracy                           0.46     10296
   macro avg       0.60      0.37      0.28     10296
weighted avg       0.57      0.46      0.33     10296



# WORD2VEC

In [31]:
# Non Stemmer

In [32]:
sentences = [nltk.word_tokenize(sentence) for sentence in data['Sentence'].tolist()]
no_stemming = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

In [33]:
current_model = no_stemming
X = np.zeros((len(data["Sentence"]), current_model.wv.vector_size), dtype="float32")
y = data["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [34]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train.tolist(), y_train.tolist())

GaussianNB()

In [35]:
pred = model.predict(X_test.tolist())

C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:453: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /


In [36]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4638
           1       0.00      0.00      0.00      4443
           2       0.00      0.00      0.00      1215

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.20      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:453: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Us

In [37]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train.tolist(), y_train.tolist())

DecisionTreeClassifier()

In [38]:
pred = model.predict(X_test.tolist())

In [39]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4638
           1       0.00      0.00      0.00      4443
           2       0.00      0.00      0.00      1215

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.20      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(X_train.tolist(), y_train.tolist())

RandomForestClassifier()

In [41]:
pred = model.predict(X_test.tolist())

In [42]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4638
           1       0.00      0.00      0.00      4443
           2       0.00      0.00      0.00      1215

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.20      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
#köklere ayrılmış hali

In [44]:
from TurkishStemmer import TurkishStemmer
turkStem = TurkishStemmer()
all_stem_lists = []

for word_group in sentences:
    output_stems = []
    for word in word_group:
        output_stems.append(turkStem.stem(word))
    all_stem_lists.append(output_stems)

In [45]:
all_stem_lists

[['ulan',
  'wifiye',
  'bağl',
  'ben',
  'on',
  'bağlıyken',
  'turkcell',
  'internet',
  'pake',
  'bitt',
  'di',
  'nasıl',
  'mesaj',
  'atabilir',
  'ban',
  'ya',
  'on',
  'da',
  'mı',
  'ödeyel'],
 ['dk',
  'gb',
  'internet',
  'mb',
  'sade',
  'kaşar',
  'turkcell',
  'de',
  'düşer',
  'oç',
  'çocuk'],
 ['ayrı',
  'turkcell',
  'superonlin',
  'reklam',
  'kadar',
  'da',
  'kötü',
  'bir',
  'reklam',
  'görmemiş'],
 ['turkcell', 'çok', 'paha', 'ya'],
 ['turkcell', 'kaş', 'internet', 'cekmiyor'],
 ['turkcell',
  'allah',
  'bela',
  'vers',
  'demek',
  'ister',
  'am',
  'demice',
  'deme',
  'farze',
  'nasıl',
  'bir',
  'internet',
  'bu',
  'ya',
  'yürek',
  'şişt'],
 ['can', 'cehenne', 'turkcell'],
 ['turkcell',
  'yönetim',
  'elin',
  'geçiren',
  'akp',
  'hük',
  'abone',
  'seç',
  'propaganda',
  'mesaj',
  'gönderiyor'],
 ['turkcell', 'şerefsiz', 'aksin', 'iddia', 'e', 'turkcell'],
 ['turkcell', 'elim', 'ayak', 'çektik', 'iyi', 'olt'],
 ['sen', 'turkcel

In [46]:
stemming = Word2Vec(sentences= all_stem_lists, vector_size=100, window=5, min_count=1, workers=4)

In [47]:
current_model = stemming
X = np.zeros((len(data["Sentence"]), current_model.wv.vector_size), dtype="float32")
y = data["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [48]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train.tolist(), y_train.tolist())

GaussianNB()

In [49]:
pred = model.predict(X_test.tolist())

C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:453: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /


In [50]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4675
           1       0.00      0.00      0.00      4493
           2       0.00      0.00      0.00      1128

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.21      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:453: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Us

In [51]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train.tolist(), y_train.tolist())

DecisionTreeClassifier()

In [52]:
pred = model.predict(X_test.tolist())

In [53]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4675
           1       0.00      0.00      0.00      4493
           2       0.00      0.00      0.00      1128

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.21      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(X_train.tolist(), y_train.tolist())

RandomForestClassifier()

In [55]:
pred = model.predict(X_test.tolist())

In [56]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4675
           1       0.00      0.00      0.00      4493
           2       0.00      0.00      0.00      1128

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.21      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
stemming.wv.most_similar(positive=["geliyor","gitmek"],negative=["gelmek"], topn=5)

[('öt', 0.9909766316413879),
 ('kapa', 0.9908773899078369),
 ('wifi', 0.9904953241348267),
 ('hırsız', 0.9899176955223083),
 ('ateş', 0.9897722005844116)]

# FastText

In [58]:
fasttext = FastText(sentences=sentences, vector_size=100, window=40, min_count=1)

In [59]:
current_model = fasttext
X = np.zeros((len(data["Sentence"]), current_model.wv.vector_size), dtype="float32")
y = data["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [60]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train.tolist(), y_train.tolist())

GaussianNB()

In [61]:
pred = model.predict(X_test.tolist())

C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:453: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /


In [62]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4642
           1       0.00      0.00      0.00      4492
           2       0.00      0.00      0.00      1162

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.20      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\naive_bayes.py:453: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Us

In [63]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train.tolist(), y_train.tolist())

DecisionTreeClassifier()

In [64]:
pred = model.predict(X_test.tolist())

In [65]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4642
           1       0.00      0.00      0.00      4492
           2       0.00      0.00      0.00      1162

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.20      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(X_train.tolist(), y_train.tolist())

RandomForestClassifier()

In [67]:
pred = model.predict(X_test.tolist())

In [68]:
from sklearn import metrics

expected = y_test.astype('int')
predicted = model.predict(X_test.astype('int'))
print(metrics.classification_report(expected,predicted))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62      4642
           1       0.00      0.00      0.00      4492
           2       0.00      0.00      0.00      1162

    accuracy                           0.45     10296
   macro avg       0.15      0.33      0.21     10296
weighted avg       0.20      0.45      0.28     10296



C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ersu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
